# Ollama Function Calling

There are 2 mechanism for function calling with Ollama. There is the _traditional_ Ollama method. It can also support the _OpenAI_ function calling API. I am trying both in this instance.

First, let's define a dummy function that can get the weather.

In [6]:
def get_weather(location: str)->str:    
    '''Get current temperature for a given location.'''
    print(f"Getting weather for: {location}")
    return "Temperature is 10 degrees"

Next, let's create a dictionary of available functions.

In [7]:
available_functions = {
    "get_weather": get_weather
}

## Ollama Method
Here is function calling using the traditional Ollama method. Note that the initial call does not actually _invoke_ the function. This has to happen outside of the `ollama.chat` call. So you need to parse the response and then make the function call.

In [8]:
import ollama
response = ollama.chat(
    model='llama3.2:latest',
    messages=[{'role': 'user', 'content':
        'What is the weather in Toronto?'}],

		# provide a weather checking tool to the model
    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_weather',
        'description': 'Get the current weather for a location',
        'parameters': {
          'type': 'object',
          'properties': {
            'location': {
              'type': 'string',
              'description': 'The name of the city, country. For example, London, United Kingdom.',
            },
          },
          'required': ['location'],
        },
      },
    },
  ],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_weather', 'arguments': {'location': 'Toronto'}}}]


In [9]:
for tool in response["message"]["tool_calls"]:
    function_to_call = available_functions[tool['function']['name']]
    function_args = tool['function']['arguments']
    function_response = function_to_call(**function_args)
    print(function_response)

Getting weather for: Toronto
Temperature is 10 degrees


## OpenAI Method
Now let's switch to the OpenAI API. If you want to develop and test your code on local host and then deploy to OpenAI, this is a much better mechanim. Again, the initial call only returns an object for tool calling. You would have to parse the result and actually call the function.

In [10]:
import json
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. London, United Kingdom"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(completion.choices[0].message.tool_calls)
function_to_call = available_functions[completion.choices[0].message.tool_calls[0].function.name]
function_args = json.loads(completion.choices[0].message.tool_calls[0].function.arguments)
function_response = function_to_call(**function_args)
print(function_response)

[ChatCompletionMessageToolCall(id='call_lg0dj7f5', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function', index=0)]
Getting weather for: Paris, France
Temperature is 10 degrees


## Upload to Cloudinary 
Now, let's enhance the function to allow a user to upload to Cloudinary. It assumes that the Cloudinary API credentials are loaded at the time of initializing the Jupyter notebook.

In [11]:
import cloudinary.uploader

def upload_to_cloudinary(url: str)->str:
    '''
    Upload the image at the url to Cloudinary
    '''
    print(f"Uploading {url}")
    resp = cloudinary.uploader.upload(
        url,
        public_id='test-image',
        unique_filename=False,
        overwrite=True
    )
    return resp

# also add this to list of available tools
available_functions['upload_to_cloudinary']=upload_to_cloudinary

In [13]:
# let's use OpenAI function
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. London, United Kingdom"
                    }
                },
                "required": [
                    "location"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "upload_to_cloudinary",
            "description": "Upload an image url to Cloudinary.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "image url to be used for Cloudinary upload"
                    }
                },
                "required": [
                    "url"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

completion = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[{"role": "user", "content": "Upload this image to Cloudinary: https://cdn.pixabay.com/photo/2020/09/20/16/50/big-buddha-5587706_1280.jpg"}],
    tools=tools
)
print(completion.choices[0].message.tool_calls)
function_to_call = available_functions[completion.choices[0].message.tool_calls[0].function.name]
function_args = json.loads(completion.choices[0].message.tool_calls[0].function.arguments)
function_response = function_to_call(**function_args)

[ChatCompletionMessageToolCall(id='call_ins2rpcm', function=Function(arguments='{"url":"https://cdn.pixabay.com/photo/2020/09/20/16/50/big-buddha-5587706_1280.jpg"}', name='upload_to_cloudinary'), type='function', index=0)]
Uploading https://cdn.pixabay.com/photo/2020/09/20/16/50/big-buddha-5587706_1280.jpg
{'asset_id': 'f6f23ec8b8037c7b476d68cccc00e490', 'public_id': 'test-image', 'version': 1737481322, 'version_id': '7cf92ee22bc1c6287c83bcc3458d90ed', 'signature': '95ec662d1935497b674ca1a5f956c3f1f690219d', 'width': 1024, 'height': 1280, 'format': 'jpg', 'resource_type': 'image', 'created_at': '2024-01-16T21:50:27Z', 'tags': [], 'pages': 1, 'bytes': 168780, 'type': 'upload', 'etag': '8e0cd6d849a93acd1b2bb51a4a3cf2d4', 'placeholder': False, 'url': 'http://res.cloudinary.com/dbmataac4/image/upload/v1737481322/test-image.jpg', 'secure_url': 'https://res.cloudinary.com/dbmataac4/image/upload/v1737481322/test-image.jpg', 'folder': '', 'access_mode': 'public', 'metadata': {'destination': [

In [14]:
if function_response.get('secure_url'):
    print(f"Image uploaded to: {function_response.get('secure_url')}")

Image uploaded to: https://res.cloudinary.com/dbmataac4/image/upload/v1737481322/test-image.jpg
